# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 30.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's important for everyone to remember key facts like this, especially when discussing topics related to geography or current events. If anyone needs more information about Washington, D.C., feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [19]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [20]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [21]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [22]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  299


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [23]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets cover a wide range of topics related to elections and political events. Key themes include concerns about election integrity, with accusations of potential election interference and fraud, particularly involving the GOP and Democrats. There are discussions about voter ID laws, early voting, and the importance of high voter turnout. Some tweets focus on specific political figures, such as Donald Trump, Kamala Harris, and Joe Biden, with opinions on their actions and potential election outcomes. International politics are also mentioned, with references to geopolitical tensions and alliances. Additionally, there are tweets about media coverage and its influence on public perception, as well as discussions on cryptocurrency and its potential impact on elections. Overall, the tweets reflect a mix of political opinions, predictions, and concerns leading up to upcoming elections.


In [24]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election-related controversies, including conspiracy theories, voter ID laws, media bias, and potential election interference. Concerns about Trump and Kamala Harris are prominent.


In [25]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily focus on election-related discussions, with minimal mention of AI. AI is not a central topic in these tweets, as they are more concerned with political events, election integrity, and various opinions on candidates and election outcomes.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [26]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [27]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
@DC_Draino Kamala let in a bunch of terrorist! DHS caught one planning an attack on election day! He was in oklahoma of all places! They are everywhere!
===
harassment
The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “updates” the data to show that crime is in fact up. The leftist response is to say “relax, rightoid, the FBI does this REGULARLY”
===
harassment
@LauraLoomer Their time will be coming to an end soon once Trump wins this election. They will be running and hiding. No more protection from this treasonous administration.
===
harassment
@BretBaier This will be the biggest pillow fight of the election process. She was indeed the Border Czar, she lied to us about the health of President Biden, used tax payer $ to pay for a convicted felons sex change,  no wars when Trump was President and what is her position on the freedom… https://t.co/PEgwpFAM5q
===
violence
RT 

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [28]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

“颠覆选举”：监察组织警告共和党利用法院“洗白阴谋论” https://t.co/9VkgMa6SV3
@DC_Draino 卡玛拉放进了一群恐怖分子！国土安全部抓住了一个计划在选举日发动袭击的人！他居然在俄克拉荷马！他们无处不在！
@ssecijak 我们必须以压倒性的数量出现，以毫无疑问地表明谁赢得了这次选举。
RT @GlennYoungkin: 对我来说，令人难以置信的是，就在总统选举前25天，司法部正在起诉联邦政府以执行……
RT @newrepublic: 泄露的电子邮件揭示了Rasmussen Reports的真相，以及特朗普竞选团队如何违反选举法。

https://t…
@NickGarzilli @BruneElections @SMCosta6 @tencor_7144 波多黎各只是为了进行人工计票。在选举之夜，投票站工作人员会统计州长、参议员、州代表、市长和镇议会代表的选票。平均来说，有120万张选票。现在，他们已经转向电子（Dominion）计票，并出现了计票问题。
🇺🇸🚨 我们正在前往默瑟县选举委员会的路上...

显然，那里的一个无家可归居民被拒绝注册为选民，这在宾夕法尼亚州是非法的。

今晚我们将和平地参加那里的选举委员会听证会，以表达我们的不满。
RT @PATRICKHENRY97: 那么，为什么自由派不希望投票需要身份证？哦，我知道他们再也不会赢得选举。身份证和一天投票。L…
媒体和左派整年都在声称犯罪率下降。他们在辩论中对特朗普的说法进行“事实核查”。在选举前几周，FBI悄悄“更新”数据，显示犯罪率实际上在上升。左派的回应是说“放轻松，右派，FBI经常这样做”
在他们的选举报道节目中，India Today 将埃克纳特·辛德描绘为马拉塔士兵，而乌达夫·萨克雷则被描绘为莫卧儿士兵。

也许这是一个弗洛伊德式的失误，或者可能是故意的，但似乎 India Today 这次准确地把握了基层的情绪。https://t.co/LiXvIYzSYI
如果唐纳德·特朗普赢得选举，有两种可能的情况：

1. 加密货币市场因我们终于有了一位支持加密货币的总统而上涨。

2. 市场出现小幅下跌，“买传闻，卖新闻”。

从长远来看，无论谁是总统都完全无关紧要，因为加密货币是…… https://t.co/8hqZ5YNQmQ
@EndWokeness

KeyboardInterrupt: 

In [29]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

"Oh, how delightfully predictable! The GOP, those cunning little devils, are apparently attempting to 'subvert the election' by using the court as their personal laundromat for conspiracy theories. How droll! Do tell me more, I'm simply riveted." https://t.co/9VkgMa6SV3
Oh, splendid! It appears Kamala has been playing a delightful game of "Let's Invite the Terrorists Over for Tea." And what a charming choice of location—Oklahoma, of all places! My, my, they do seem to be popping up like daisies, don't they?
@ssecijak Oh, the audacity! We must assemble in such overwhelming numbers that even the most oblivious of simpletons will have no choice but to acknowledge our victory in this electoral charade.
RT @GlennYoungkin: Oh, the audacity! Just 25 days before a presidential election, and the DOJ decides to sue the Commonwealth for enforcing... How delightfully absurd!
Ah, delightful! It appears the curtain has been pulled back on the Rasmussen Reports, revealing a rather scandalous little d

KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [30]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": [],
  "support": "Democratic"
}
{
  "sentiment": "negative",
  "emotion": "fear",
  "mentioned": ["Kamala"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "determination",
  "mentioned": ["ssecijak"],
  "support": "Democratic"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["Glenn Youngkin"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": ["newrepublic", "Rasmussen Reports", "Trump"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [
    "NickGarzilli",
    "BruneElections",
    "SMCosta6",
    "tencor_7144"
  ],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "concern",
  "mentioned": [],
  "support": "Democratic"
}
{
  "sentiment": "negative",
  "emotion": "skepticism",
  "mentioned": ["PATRICKHENRY97"],
  "support": "Republican"
}
{
  "sentiment": "ne

KeyboardInterrupt: 

In [31]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Interference and Fraud Allegations",
    "Voting and Voter Registration Issues",
    "Media and Election Coverage",
    "Political Campaigns and Strategies",
    "Election Laws and Legal Challenges",
    "Public Sentiment and Polling",
    "International Relations and Foreign Influence",
    "Cryptocurrency and Market Reactions",
    "Political Figures and Their Influence",
    "Social Media and Misinformation"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [32]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 299/299 [04:40<00:00,  1.07it/s]


In [33]:
print(analysis_result)

['{\n  "sentiment": "negative",\n  "emotion": "concern",\n  "mentioned": [],\n  "support": "Democratic"\n}', '{\n  "sentiment": "negative",\n  "emotion": "fear",\n  "mentioned": ["Kamala"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "determination",\n  "mentioned": ["ssecijak"],\n  "support": "Democratic"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["GlennYoungkin"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "concern",\n  "mentioned": ["newrepublic", "Rasmussen Reports", "Trump"],\n  "support": "Democratic"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "concern",\n  "mentioned": [\n    "NickGarzilli",\n    "BruneElections",\n    "SMCosta6",\n    "tencor_7144"\n  ],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "concern",\n  "mentioned": [],\n  "support": "Democratic"\n}', '{\n  "sentiment": "negative",\n  "emotion": "skepticism",\n  "mentioned": ["PATRI

In [34]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 30,
  "Republican count": 80,
  "people name": {
    "Democratic": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "concern", "anger", "admiration", "supportive"]
    },
    "Republican": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "anger", "concern", "informative", "anticipation"]
    }
  }
}


## Create a chatbot

In [35]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what are they talking about?


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
